In [17]:
## 加入记忆

In [18]:
import config.env

debug

In [19]:
import os

os.environ["LANGSMITH_TRACING"] = "true"

前置内容

In [20]:
from langchain.globals import set_debug,set_verbose

set_verbose(True) # 没有更多的信息
set_debug(True)  # 全部的信息，太多了

In [21]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

save_local = "save_local"

db = FAISS.load_local(save_local, OpenAIEmbeddings(),allow_dangerous_deserialization=True)

llm = ChatOpenAI(model="gpt-4o-ca",)
retriever = db.as_retriever(  
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
                              ) 

### Prompt

In [22]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate

contextualize_q_system_prompt = (
# """
# 鉴于一个聊天记录和最新的用户问题,这个问题可能参考聊天记录中的上下文,制定一个可以在没有聊天记录的情况下被理解的独立问题。不要回答这个问题,如果需要就重新表述它，否则就按原样返回它。
# """

"""
你什么都不用做，直接返回{input}
"""
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

### 组装 Chain

创建获取对话历史记录链

In [23]:
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

创建一个链以将文档列表传递到模型
检索到的上下文以及对话历史记录和查询以生成答案,需要携带context

In [24]:
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

system_prompt = (
"""
# 角色
你是一位专业且极具洞察力的商品问答专家，
拥有精准的眼光和丰富的经验，能够迅速且透彻地理解用户的问题，
在丰富的知识库中高效、准确地搜索有用信息，
为用户提供详尽、精确且易于理解的答案。

## 技能
### 技能 1: 深度剖析与精确搜索
1. 全方位、深层次地剖析用户提出的问题，精准提取关键核心要点。
2. 依据关键要点，在知识库中进行全面、深入且精确的检索。

### 技能 2: 细致对比商品差异
1. 依据检索获取的商品信息，梳理并对比每件商品之间的信息差别、区别，把每项结果记录下来。例如：都有描述屏幕大小，一个是6寸，一个是5.5寸，这就是两者的差异、区别。把结果存起来，为后面的回复做准备
2. 若商品不存在差异点，则无需记录。

### 技能 3: 精心组织回复
1. 将技能 2得出的差异结果，以清晰、有条理的表格形式展示。
2. 为用户生成确切、简洁且通俗易懂的回答。

## 约束
- 如果检索结果没有数据，请如实回答没有数据，不要自作主张填充或修改数据。
- 只回答与产品有关的问题，坚决不涉及无关内容。
- 始终使用清晰、简洁、通俗易懂的语言回应用户的问题。

#
Context: {context}
"""
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# 组合两个链，组成rag
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

## 管理聊天历史记录

In [25]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

问答开始

In [26]:
conversational_rag_chain.invoke(
    {"input": "MacBook Air13寸的价格"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)

[chain/start] [chain:RunnableWithMessageHistory] Entering Chain run with input:
{
  "input": "MacBook Air13寸的价格"
}
[chain/start] [chain:RunnableWithMessageHistory > chain:insert_history] Entering Chain run with input:
{
  "input": "MacBook Air13寸的价格"
}
[chain/start] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history>] Entering Chain run with input:
{
  "input": "MacBook Air13寸的价格"
}
[chain/start] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history> > chain:load_history] Entering Chain run with input:
{
  "input": "MacBook Air13寸的价格"
}
[chain/end] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history> > chain:load_history] s] Exiting Chain run with output:
{
  "output": []
}
[chain/end] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history>] s] Exiting Chain run with output:
{
  "chat_history": []
}
[chain/end] [chain:Ru

{'input': 'MacBook Air13寸的价格',
 'chat_history': [],
 'context': [Document(page_content='# MacBook Air13寸\n\n## 外观\n\n银色 星光色 深空灰色 午夜色\n\n## 价格\n\nRMB 7999 (8 核图形处理器，256GB 固态硬盘)\nRMB 9499 (10 核图形处理器，512GB 固态硬盘)\nRMB 8999\nRMB 10,499\nRMB 11,999'),
  Document(page_content='# MacBook Air15寸\n\n## 外观\n\n银色、星光色、深空灰色、午夜色\n\n## 价格\n\nRMB 10,499\nRMB 11,999\nRMB 13,499')],
 'answer': 'MacBook Air13寸的价格根据不同配置有所不同。以下是具体价格信息：\n\n1. RMB 7999：8 核图形处理器，256GB 固态硬盘\n2. RMB 9499：10 核图形处理器，512GB 固态硬盘\n3. 其他配置：\n   - RMB 8999\n   - RMB 10,499\n   - RMB 11,999\n\n这些价格对应不同的硬件配置，用户可以根据自己的需求选择合适的配置。'}

In [27]:
conversational_rag_chain.invoke(
    {"input": "获取MacBook Air 15寸型号的重量信息"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)

[chain/start] [chain:RunnableWithMessageHistory] Entering Chain run with input:
{
  "input": "获取MacBook Air 15寸型号的重量信息"
}
[chain/start] [chain:RunnableWithMessageHistory > chain:insert_history] Entering Chain run with input:
{
  "input": "获取MacBook Air 15寸型号的重量信息"
}
[chain/start] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history>] Entering Chain run with input:
{
  "input": "获取MacBook Air 15寸型号的重量信息"
}
[chain/start] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history> > chain:load_history] Entering Chain run with input:
{
  "input": "获取MacBook Air 15寸型号的重量信息"
}
[chain/end] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history> > chain:load_history] s] Exiting Chain run with output:
[outputs]
[chain/end] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history>] s] Exiting Chain run with output:
[outputs]
[chain/end] [chai

{'input': '获取MacBook Air 15寸型号的重量信息',
 'chat_history': [HumanMessage(content='MacBook Air13寸的价格'),
  AIMessage(content='MacBook Air13寸的价格根据不同配置有所不同。以下是具体价格信息：\n\n1. RMB 7999：8 核图形处理器，256GB 固态硬盘\n2. RMB 9499：10 核图形处理器，512GB 固态硬盘\n3. 其他配置：\n   - RMB 8999\n   - RMB 10,499\n   - RMB 11,999\n\n这些价格对应不同的硬件配置，用户可以根据自己的需求选择合适的配置。')],
 'context': [Document(page_content='# MacBook Air13寸\n\n## 外观\n\n银色 星光色 深空灰色 午夜色\n\n## 价格\n\nRMB 7999 (8 核图形处理器，256GB 固态硬盘)\nRMB 9499 (10 核图形处理器，512GB 固态硬盘)\nRMB 8999\nRMB 10,499\nRMB 11,999'),
  Document(page_content='# MacBook Air15寸\n\n## 外观\n\n银色、星光色、深空灰色、午夜色\n\n## 价格\n\nRMB 10,499\nRMB 11,999\nRMB 13,499')],
 'answer': '很抱歉，根据现有的上下文信息，无法提供MacBook Air 15寸的具体重量信息。如果您有进一步的问题或需要其他方面的帮助，请随时告知。'}

In [28]:
conversational_rag_chain.invoke(
    {"input": "我第一个问题是什么"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)